# Chapter 4: Libraries

## Intro

### Download files

In [ ]:
!git clone https://github.com/delima87/Ch4Data.git

Cloning into 'Ch4Data'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (5/5), 24.55 MiB | 25.27 MiB/s, done.


### Read video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Ch4Data/VideoKROOK.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width="640" height="480" controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


# 4.1 Computer Vision library (OpenCV)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

frameId = 100 # Change this value to pick a frame

cap = cv2.VideoCapture('/content/Ch4Data/VideoKROOK.mp4')
frameCounter =0
img2Process = np.zeros((720,512,3), np.uint8)
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    frameCounter += 1
    if frameCounter == frameId:
        img2Process = cv2.resize(frame, (720,512), interpolation = cv2.INTER_AREA)
        break
  # Break the loop
  else:
    break

img2Process_rgb = cv2.cvtColor(img2Process, cv2.COLOR_BGR2RGB)
plt.imshow(img2Process_rgb)

## 4.1.1 Light green caps

In [ ]:
# set the correct limits for r,g,b
lR = 0
lG = 0
lB = 0
hR = 0
hG = 0
hB = 0


low = (lR, lG, lB)
high = (hR, hG, hB)
im_rgb = cv2.cvtColor(img2Process, cv2.COLOR_BGR2RGB)
bin_img = cv2.inRange(im_rgb, low, high)
plt.imshow(bin_img, cmap='Greys',  interpolation='nearest')

## 4.1.2 Morphological Operations

In [ ]:
#erode kernel 2 by 2
kernel = np.ones((2,2), np.uint8)
img_erosion = cv2.erode(bin_img, kernel, iterations=1)
#dilate kernel 16 by 16
kernel2 = np.ones((16,16), np.uint8)#define kernel 2
img_dilation = cv2.dilate(img_erosion, kernel2, iterations=1)
#erode with kernel 4 by 4
#YOUR CODE >

# < YOUR CODE
plt.imshow(img_dilation, cmap='Greys',  interpolation='nearest')

# 4.2 Pointcloud library (Open3D)

## Installing Open3D

In [ ]:
!pip install open3d
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.5
    Uninstalling widgetsnbextension-3.6.5:
      Successfully uninstalled widgetsnbextension-3.6.5
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3

## 4.2.1 Visualize Point Clouds

In [ ]:
def showPCD(pcd):
  points = np.asarray(pcd.points)
  colors = None
  if pcd.has_colors():
      colors = np.asarray(pcd.colors)
  elif pcd.has_normals():
      colors = (0.5, 0.5, 0.5) + np.asarray(pcd.normals) * 0.5
  fig = go.Figure(
  data=[
    go.Scatter3d(
      x=points[:,0], y=points[:,1], z=points[:,2],
      mode='markers',
      marker=dict(size=1, color=colors)
  )
  ],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
  )
  )
  )
  fig.show()


pcd = o3d.io.read_point_cloud('/content/Ch4Data/cloud_2.ply')
showPCD(pcd)



## 4.2.2 Plane Fitting

Aangezien puntenwolken die met laserscanners worden gegenereerd gewoonlijk vrij dicht zijn, is het computationeel duur om metingen uit te voeren. Een mogelijke oplossing is het inpassen van geometrische primitieven in de puntenwolk. De belangrijkste primitieve is de eenvoudigste 3D vorm: een vlak. Open3D gebruikt hiervoor de functie `segment_plane()`. Je kan verschillende parameters doorgeven aan deze functie, maar in zijn eenvoudigste vorm heeft hij alleen een puntenwolk, een maximale afstand, het aantal minimale punten en iteraties nodig. De functie berekent dan het best passende vlak (d.w.z. het vlak met de meeste punten in de puntenwolk). Naast het vlak geeft deze functie ook de inliers of de punten die in het vlak liggen. 

### Eerste vlak

- Laad een puntenwolk naar keuze in.
- Gebruik `segment_plane()` om het best passende vlak in de puntenwolk te passen. Kies zelf een goede waarde voor de maximale afstand `distance_threshold`). Stel de andere parameters in met deze waarden:  `ransac_n = 3, num_iterations=500`.
- Bekijk het model dat je terugkrijgt. Wat zouden deze getallen betekenen?
- Gebruik de functie `select_by_index()` om alleen de inliers van dit vlak uit de oorspronkelijke puntenwolk te selecteren.
- Visualiseer de inliers en de puntenwolk. Gebruik de functie `paint_uniform_color()` om de uitschieters in te kleuren.

In [ ]:
# Fitting 1st plane
plane_model, inliers = pcd.segment_plane(distance_threshold=0.08,ransac_n=3,num_iterations=500)
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
# visualizing extracted plane
showPCD(inlier_cloud+ outlier_cloud)

### Andere vlakken
Zodra het eerste (dominante) vlak is aangebracht, kunt u op zoek gaan naar het op één na beste vlak. Volg hiervoor de volgende deze stappen:
- Nadat het dominante vlak is gedetecteerd, wordt een nieuwe puntenwolk gemaakt waarin de inliers van het eerste vlak zijn verwijderd. Gebruik daarvoor de functie `select_by_index()` en de uitschieters. filter de verkregen inliers en stel `invert = True`.
- Voer nu dezelfde functie `segment_plane()` uit op deze puntenwolk en vind het op één na beste vlak.
- Herhaal stappen één en twee om het derde, vierde of vijfde vlak te vinden.
- Visualiseer de resultaten door de gebieden telkens een andere kleur te geven.

In [ ]:

#plane 2
plane_model2, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.05,ransac_n=3,num_iterations=500)
inlier_cloud2 = outlier_cloud.select_by_index(inliers2)
inlier_cloud2.paint_uniform_color([0, 1.0, 0])
outlier_cloud2 = outlier_cloud.select_by_index(inliers2, invert=True)
showPCD(inlier_cloud2+outlier_cloud2 + inlier_cloud)

### Plane extraction on gable roof

In [ ]:
#read gable roof
gable_roof = '/content/Ch4Data/gableRoof2.ply'
roof = o3d.io.read_point_cloud(gable_roof)
showPCD(roof)

 Bepaal het snijpunt van de 2 relevante dominante vlakken om de nok van het dak te bepalen. Daartoe herhalen we een beetje wiskunde. Stel je hebt twee vlakken ${\tt \Pi_1}$ and ${\tt \Pi_2}$ met vergelijking:
$$	{\tt \Pi_1}: a_1X+b_1Y+c_1Z+d_1 = 0\nonumber \\
	{\tt \Pi_2}: a_2X+b_2Y+c_2Z+d_2 = 0
$$

Hoe bereken je het snijpunt van deze twee vlakken? De vergelijking van een rechte kan parametrisch geschreven worden:
$$
{\tt M} = {\tt M_0} + \lambda {\tt D}\nonumber \\
\begin{bmatrix}X\\Y\\Z\end{bmatrix} = \begin{bmatrix}X_0\\Y_0\\Z_0\end{bmatrix} + \lambda\begin{bmatrix}X_d\\Y_d\\Z_d\end{bmatrix}
$$
waardat ${\tt M_0}$ is een punt op de lijn en ${\tt D}$ is de richting van de lijn. Deze richting is gemakkelijk te berekenen. We weten immers dat het snijpunt van de twee vlakken in beide vlakken moet liggen. Dat betekent dat de richting ervan loodrecht moet staan op de normalen van beide vlakken:
$$
	{\tt D} \bot {\tt n_1}\\
	{\tt D} \bot {\tt n_2}
$$
Zo kan ${\tt D}$ bepaald worden als het vectorproduct van ${\tt n_1}$ en ${\tt n_2}$:
$$
{\tt D} = {\tt n_1} \times {\tt n_2}
$$
Je ziet een illustratie hier:

![](cuttingplanes.png)


We hoeven alleen maar ${\tt M_0}$ (een willekeurig punt op de lijn) te bepalen. We kunnen dit eenvoudig als volgt doen:
- Bepaal 1 coördinaat in de buurt van de lijn, Bijvoorbeeld een $(X)$ coördinaat: $X = X_0$.
- Als $X_0$ bekend is, bepaal dan $Y_0$ en $Z_0$, wetende dat het punt op de secans moet liggen, en dus aan beide vergelijkingen moet voldoen. Stel de twee vergelijkingen op en los het systeem op.

Het volledige algoritme wordt dan:
- Bepaal de normalen van de vlakken ${\tt n_1}$ en ${\tt n_2}$.
- Bepaal de richting ${\tt D} = {\tt n_1} \times {\tt n_2}$ (tip: `numpy.cross()`).
- Kies een coördinaat ${X_0}$.
- Stel het stelsel ($Y_0$ and $Z_0$) op en los op.
- Teken nu het snijpunt door $\lambda$ te variëren. Als ${\tt n_1}$ en ${\tt n_2}$ eenheidsvectoren zijn, is $\lambda$ een afstand in meters. Afhankelijk van de keuze van je startpunt ($X_0$) moet je $\lambda$ positieve en/of negatieve waarden geven. 
- Om de lijn te tekenen gebruik je de functie open3D `geometry.LineSet()`, waar je de punten moet invoeren die de lijn vormen en de array van verbindingen. Een voorbeeld van het gebruik van deze functie vindt u in deze [link](http://www.open3d.org/docs/0.7.0/tutorial/Basic/visualization.html)

In [ ]:
#YOUR CODE >

# Extract plane1


# Plane2


# Visualise

# < YOUR CODE


Punt $x0$ ligt op de snijlijn

In [ ]:
x0 =103831
#YOUR CODE >


# < YOUR CODE